# PySDR初探

## 测试连接

In [6]:
import adi
sdr = adi.Pluto('ip:192.168.2.1') # or whatever your Pluto's IP is
sdr.sample_rate = int(2.5e6)
sdr.rx()

array([ 43.+51.j,  -8.-13.j, -44.-20.j, ...,  36. -3.j,   6.-10.j,
        10. +3.j])

## 测试接收信号

## 发送QPSK信号

In [10]:
import adi
import numpy as np

# 配置ADALM-Pluto设备
sdr = adi.Pluto("ip:192.168.2.1")
sdr.sample_rate = int(1e6)
sdr.tx_rf_bandwidth = int(50e3)
sdr.tx_lo = int(435e6)
sdr.tx_hardwaregain_chan0 = -10  # 合理的增益范围
sdr.tx_cyclic_buffer = True

# 确保发送缓冲区在设备允许范围内
sdr.tx_buffer_size = 1024

# QPSK信号生成函数
def generate_qpsk(num_samples):
    bits = np.random.randint(0, 2, num_samples * 2)  # 生成随机比特
    symbols = bits[0::2] + 1j * bits[1::2]  # 形成复数符号
    return symbols

# 生成QPSK信号
num_samples = sdr.tx_buffer_size //2
qpsk_signal = generate_qpsk(num_samples)

# 确保信号幅度符合ADALM-Pluto的范围
qpsk_signal *= 2**14

# 发送QPSK信号
sdr.tx(qpsk_signal)



# 发送FM音频

In [19]:
import adi
import numpy as np
from scipy.io import wavfile

# 配置ADALM-Pluto设备
sdr = adi.Pluto("ip:192.168.2.1")
sdr.sample_rate = int(1e6)
sdr.tx_rf_bandwidth = int(0.5e3)
sdr.tx_lo = int(435e6)
sdr.tx_hardwaregain_chan0 = -10  # 合理的增益范围
sdr.tx_cyclic_buffer = True  # 设置循环模式

# 读取WAV文件并获取前30秒的音频数据
sample_rate, audio_data = wavfile.read("WildCards.wav")
audio_data = audio_data / 32768.0  # 假设16-bit PCM数据，归一化到[-1, 1]范围
seconds_to_send = 30
num_samples = sample_rate * seconds_to_send  # 计算前30秒的样本数
audio_data = audio_data[:num_samples]  # 只保留前30秒的音频数据

# FM调制函数
def fm_modulate(audio, deviation, sample_rate):
    t = np.arange(len(audio)) / sample_rate
    integral = np.cumsum(audio) / sample_rate
    return np.exp(2j * np.pi * (deviation * integral))

# 使用裁剪后的信号进行发送
modulated_signal = fm_modulate(audio_data, 75e3, sample_rate)
modulated_signal *= 2**14

# 发送FM调制信号
sdr.tx(modulated_signal)


# 清空缓存区

In [16]:
sdr.tx_destroy_buffer()